In [1]:
import os, sys, math, random, time
from datetime import datetime
import _pickle as pickle

import numpy as np
import pandas as pd

import mxnet as mx
from mxnet import autograd, gluon, nd, gpu
from mxnet.gluon import nn

from nnModels import TCN
from nnTrainer import nn_trainer

In [2]:
### The input dataset
ctx = mx.gpu(2)
with open('feature_prepare.pkl', 'rb') as f:
    [trainX_dt,trainX2_dt, trainY_dt,trainY2_dt, testX_dt, testX2_dt,testY_dt,testY2_dt] = pickle.load(f)

In [3]:
sub_train_X, sub_train_Y = nd.array(trainX_dt, ctx=ctx), nd.array(trainY_dt, ctx=ctx)
sub_valid_X, sub_valid_Y = nd.array(testX_dt, ctx=ctx), nd.array(testY_dt, ctx=ctx)
future_train_X, future_test_X = nd.array(trainY2_dt, ctx=ctx), nd.array(testY2_dt, ctx=ctx)

sub_train_nd = gluon.data.ArrayDataset(sub_train_X, future_train_X, sub_train_Y)

In [4]:
model1 = TCN()
#choose parameters
batch_size= 16
n_epochs=100
"""
The model training
"""
### The model parameters
abs_loss = gluon.loss.L1Loss()
L2_loss = gluon.loss.L2Loss()
huber_loss = gluon.loss.HuberLoss()
initializer = mx.initializer.MSRAPrelu()
optimizer = 'adam';
optimizer_params = {'learning_rate': 0.005}

trainer_params_list = {'batch_size': batch_size,'epoch_num':n_epochs,
                'loss_func': abs_loss, 'initializer': initializer,
                'optimizer':optimizer, 'optimizer_params':optimizer_params}
train_mark='testing'
nn_trainer(train_mark, model1, sub_train_nd, sub_valid_X,future_test_X, sub_valid_Y, trainer_params_list=trainer_params_list, ctx=ctx)

  0%|          | 0/100 [00:00<?, ?it/s]


MXNetError: [15:02:58] src/operator/nn/convolution.cc:138: Check failed: dilated_ksize_x <= AddPad(dshape[2], param_.pad[0]) (5 vs. 2) kernel size exceed input

Stack trace returned 10 entries:
[bt] (0) /usr/local/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2e688a) [0x7f689c76388a]
[bt] (1) /usr/local/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2e6ea1) [0x7f689c763ea1]
[bt] (2) /usr/local/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x5608b5) [0x7f689c9dd8b5]
[bt] (3) /usr/local/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2618ada) [0x7f689ea95ada]
[bt] (4) /usr/local/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2622fcd) [0x7f689ea9ffcd]
[bt] (5) /usr/local/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2555439) [0x7f689e9d2439]
[bt] (6) /usr/local/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(MXImperativeInvokeEx+0x6f) [0x7f689e9d2a2f]
[bt] (7) /usr/local/anaconda3/lib/python3.6/lib-dynload/../